In [69]:
!poetry env use 3.12
!poetry env activate

Using virtualenv: C:\Users\04n\AppData\Local\pypoetry\Cache\virtualenvs\corn-leaf-disease-classifier-davY0eoL-py3.12
"C:\Users\04n\AppData\Local\pypoetry\Cache\virtualenvs\corn-leaf-disease-classifier-davY0eoL-py3.12\Scripts\activate.bat"


In [70]:
!poetry install
!poetry lock
#!poetry run pip freeze
    # ipykernel
    # keras
    # matplotlib
    # numpy
    # ruff
    # seaborn
    # tensorflow

Installing dependencies from lock file

No dependencies to install or update
Resolving dependencies...


# Setup Imports

In [71]:
import keras
import numpy as np

In [72]:
from natsort import natsorted
from numpy._typing._array_like import NDArray
from pathlib import Path
from PIL import Image, ImageEnhance
from typing import Any

IMAGE_SIZE: tuple[int, int] = (64, 64)
IMAGE_SUFFIXES: set[str] = {'.jpeg', '.jpg', '.png'}

# Load Datasets

In [73]:
ROOT_PATH: Path = Path.cwd().parent
TRAIN_PATH: Path = ROOT_PATH / 'data' / 'train'
TEST_PATH: Path = ROOT_PATH / 'data' / 'test'

In [74]:
def load_image_from_path(image_path: Path) -> list[int]:
    image: Image.Image = Image.open(image_path).convert('RGB').resize(IMAGE_SIZE)
    image = ImageEnhance.Color(image).enhance(2.0)

    image_flat: NDArray[Any] = np.asarray(image, dtype="int64").flatten()

    ret: list[int] = [int(i) for i in image_flat]
    return ret

In [75]:
def get_dataset_classes(dataset_path: Path) -> list[str]:
    return [d.name for d in dataset_path.iterdir() if d.is_dir()]

# Load Model from File

In [76]:
MODEL_PATH: Path = Path.cwd() / 'final' / 'model.keras'

In [77]:
model = keras.saving.load_model(MODEL_PATH)

# Predict Image

In [78]:
def predict_image(image_path: Path) -> str:
    image_raw: list[int] = load_image_from_path(image_path)
    image: NDArray[Any] = np.array(image_raw).reshape(1, IMAGE_SIZE[0], IMAGE_SIZE[1], 3)

    pred = model.predict(image)
    pred_class = np.argmax(pred, axis = 1) 
    prediction: list[str] = [get_dataset_classes(TRAIN_PATH)[i] for i in pred_class]

    return prediction[0]

# Predict Images

In [79]:
def predict_images(image_directory: Path) -> list[tuple[str, str]]:
    image_paths: list[Path] = [image_path for image_path in image_directory.iterdir() if image_path.is_file() and image_path.suffix in IMAGE_SUFFIXES]
    images: list[Image.Image] = [load_image_from_path(image_path) for image_path in image_paths]

    image_arrays = np.array(images).reshape(-1, IMAGE_SIZE[0], IMAGE_SIZE[1], 3)

    pred = model.predict(image_arrays)
    pred_classes = np.argmax(pred, axis=1)
    predictions = [get_dataset_classes(TRAIN_PATH)[i] for i in pred_classes]

    zipped_predictions: list[tuple[Path, str]] = [(image_path, str(prediction)) for image_path, prediction in zip(image_paths, predictions)]
    
    return natsorted([(p[0].name, p[1]) for p in zipped_predictions])

# Demo

In [80]:
predict_image(TRAIN_PATH / 'Blight' / 'b23.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


'Blight'

In [83]:
predict_images(TRAIN_PATH / 'Gray_Leaf_Spot')

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


[('g0.jpeg', 'Blight'),
 ('g1.jpeg', 'Blight'),
 ('g2.jpeg', 'Gray_Leaf_Spot'),
 ('g3.jpeg', 'Blight'),
 ('g4.jpeg', 'Healthy'),
 ('g5.jpeg', 'Blight'),
 ('g6.jpeg', 'Blight'),
 ('g7.jpeg', 'Blight'),
 ('g8.jpeg', 'Blight'),
 ('g9.jpeg', 'Common_Rust'),
 ('g10.jpeg', 'Common_Rust'),
 ('g11.jpeg', 'Blight'),
 ('g12.jpeg', 'Gray_Leaf_Spot'),
 ('g13.jpeg', 'Blight'),
 ('g14.jpeg', 'Gray_Leaf_Spot'),
 ('g15.jpeg', 'Gray_Leaf_Spot'),
 ('g16.jpeg', 'Gray_Leaf_Spot'),
 ('g17.jpeg', 'Gray_Leaf_Spot'),
 ('g18.jpeg', 'Blight'),
 ('g19.jpeg', 'Common_Rust'),
 ('g20.jpeg', 'Gray_Leaf_Spot'),
 ('g21.jpeg', 'Gray_Leaf_Spot'),
 ('g22.jpeg', 'Blight'),
 ('g23.jpeg', 'Gray_Leaf_Spot'),
 ('g24.jpeg', 'Gray_Leaf_Spot'),
 ('g25.jpeg', 'Blight'),
 ('g26.jpeg', 'Gray_Leaf_Spot'),
 ('g27.jpeg', 'Blight'),
 ('g28.jpeg', 'Gray_Leaf_Spot'),
 ('g29.jpeg', 'Blight'),
 ('g30.jpeg', 'Common_Rust'),
 ('g31.jpeg', 'Gray_Leaf_Spot'),
 ('g32.jpeg', 'Gray_Leaf_Spot'),
 ('g33.jpeg', 'Blight'),
 ('g34.jpeg', 'Gray_Leaf_Sp

In [82]:
predict_images(TEST_PATH)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


[('0.jpeg', 'Healthy'),
 ('1.jpeg', 'Healthy'),
 ('2.jpeg', 'Healthy'),
 ('3.jpeg', 'Healthy'),
 ('4.jpeg', 'Healthy'),
 ('5.jpeg', 'Healthy'),
 ('6.jpeg', 'Healthy'),
 ('7.jpeg', 'Healthy'),
 ('8.jpeg', 'Healthy'),
 ('9.jpeg', 'Healthy'),
 ('10.jpeg', 'Healthy'),
 ('11.jpeg', 'Healthy'),
 ('12.jpeg', 'Healthy'),
 ('13.jpeg', 'Healthy'),
 ('14.jpeg', 'Healthy'),
 ('15.jpeg', 'Healthy'),
 ('16.jpeg', 'Healthy'),
 ('17.jpeg', 'Healthy'),
 ('18.jpeg', 'Healthy'),
 ('19.jpeg', 'Healthy'),
 ('20.jpeg', 'Healthy'),
 ('21.jpeg', 'Healthy'),
 ('22.jpeg', 'Healthy'),
 ('23.jpeg', 'Healthy'),
 ('24.jpeg', 'Healthy'),
 ('25.jpeg', 'Healthy'),
 ('26.jpeg', 'Healthy'),
 ('27.jpeg', 'Healthy'),
 ('28.jpeg', 'Healthy'),
 ('29.jpeg', 'Healthy'),
 ('30.jpeg', 'Healthy'),
 ('31.jpeg', 'Healthy'),
 ('32.jpeg', 'Healthy'),
 ('33.jpeg', 'Healthy'),
 ('34.jpeg', 'Healthy'),
 ('35.jpeg', 'Healthy'),
 ('36.jpeg', 'Healthy'),
 ('37.jpeg', 'Healthy'),
 ('38.jpeg', 'Healthy'),
 ('39.jpeg', 'Healthy'),
 ('40.jpeg